# Lineage guesser demo

## Prerequisites

Install `bread` : see https://ninivert.github.io/bread/readme.html#installation

Install packages :

```
# Optional (for this notebook, pandas is used.)
pip install pandas
```

Get example data : download and extract https://drive.google.com/file/d/1XcXTDPyDiBBlLeQpNFmugn3O6xVFfleU/view

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bread.algo.lineage import LineageGuesserBudLum, LineageGuesserExpansionSpeed, accuracy
from bread.data import Lineage, Segmentation, Microscopy

Load the data

Note : the segmentations loaded here have been corrected manually. Expect lesser performance for uncorrected segmentations obtained from YeaZ.

In [3]:
# using compressed numpy files (takes less space, but needs to preprocessed to be stored in this format)
# from glob import glob
# segmentation = Segmentation.from_npzs(sorted(glob('data/segmentations/colony003/*.npz')))
# microscopy_budneck = Microscopy.from_npzs(sorted(glob('data/microscopy_budneck/colony003/*.npz')))

# using raw microscopy (takes more space, but can be exported straight from Fiji)
segmentation = Segmentation.from_h5('../data/colony003_segmentation.h5')
microscopy_budneck = Microscopy.from_tiff('../data/colony003_GFP.tif')

## Tip : generate an empty lineage file for manual completion

In [4]:
segmentation.find_buds().save_csv('outputs/colony003_lineage_empty.csv')

## Lineage guesser using bud expansion velocities

This method needs only the segmentation file from YeaZ.

Expect ~75% accuracy.

In [5]:
?LineageGuesserExpansionSpeed

Init signature:
LineageGuesserExpansionSpeed(
    segmentation: bread.data._data.Segmentation,
    nn_threshold: float = 8,
    flexible_nn_threshold: bool = False,
    num_frames_refractory: int = 0,
    num_frames: int = 5,
    ignore_dist_nan: bool = True,
    bud_distance_max: float = 7,
) -> None
Docstring:     
Guess lineage relations by maximizing the expansion velocity of the bud with respect to the candidate parent.

Parameters
----------
segmentation : Segmentation
nn_threshold : float, optional
        cell masks separated by less than this threshold are considered neighbors, by default 8.0
flexible_nn_threshold : bool, optional
        If no nearest neighbours are found within the given threshold, try to find the closest one, by default False
num_frames_refractory : int, optional
        After a parent cell has budded, exclude it from the parent pool in the next frames.
        It is recommended to set it to a low estimate, as high values will cause mistakes to propagate in

In [6]:
guesser_expspeed = LineageGuesserExpansionSpeed(
	segmentation=segmentation,
	# see docstring for more options
)
guesser_expspeed

LineageGuesserExpansionSpeed(segmentation=Segmentation(num_frames=181, frame_height=575, frame_width=625), nn_threshold=8, flexible_nn_threshold=False, num_frames_refractory=0, num_frames=5, ignore_dist_nan=True, bud_distance_max=7)

In [7]:
lineage_expspeed = guesser_expspeed.guess_lineage()

/home/niels/Documents/bread/src/bread/algo/lineage/_lineage.py:189: BreadWarning: cell #3 does not have nearest neighbours with a distance less than 8, and flexible_threshold is False.
  warnings.warn(BreadWarning(f'cell #{nearest_neighbours_of} does not have nearest neighbours with a distance less than {self.nn_threshold}, and flexible_threshold is {self.flexible_nn_threshold}.'))
/home/niels/Documents/bread/src/bread/algo/lineage/_lineage.py:581: LineageWarning: Unable to determine distance due to invalid contour, got exception InvalidContourException('Unable to extract a contour from the mask. Did you check visually if the mask is connected, or large enough (found 2 nonzero pixels) ?'). Computation for frame #167, bud #73, candidate #53 skipped.
  warnings.warn(LineageWarning(f'Unable to determine distance due to invalid contour, got exception {repr(e)}. Computation for frame #{time_id_}, bud #{bud_id}, candidate #{parent_id} skipped.'))
/home/niels/Documents/bread/src/bread/algo/li

## Lineage guesser using budneck marker

This method needs the segmentation file from YeaZ and the budneck movie (GFP marker).

Expect near perfect (>95%) accuracy.

In [8]:
?LineageGuesserBudLum

Init signature:
LineageGuesserBudLum(
    budneck_img: bread.data._data.Microscopy,
    segmentation: bread.data._data.Segmentation,
    nn_threshold: float = 8,
    flexible_nn_threshold: bool = False,
    num_frames_refractory: int = 0,
    num_frames: int = 5,
    offset_frames: int = 0,
    kernel_N: int = 30,
    kernel_sigma: int = 1,
) -> None
Docstring:     
Guess lineage relations by looking at the budneck marker intensity along the contour of the bud.

Parameters
----------
segmentation : Segmentation
budneck_img : Microscopy
nn_threshold : float, optional
        Cell masks separated by less than this threshold are considered neighbors, by default 8.0
flexible_nn_threshold : bool, optional
        If no nearest neighbours are found within the given threshold, try to find the closest one, by default False
num_frames_refractory : int, optional
        After a parent cell has budded, exclude it from the parent pool in the next frames.
        It is recommended to set it to a lo

In [9]:
guesser_budlum = LineageGuesserBudLum(
	segmentation=segmentation,
	budneck_img=microscopy_budneck,
	# see docstring for more options
)
guesser_budlum

LineageGuesserBudLum(budneck_img=Microscopy(num_frames=181, frame_height=575, frame_width=625), segmentation=Segmentation(num_frames=181, frame_height=575, frame_width=625), nn_threshold=8, flexible_nn_threshold=False, num_frames_refractory=0, num_frames=5, offset_frames=0, kernel_N=30, kernel_sigma=1)

In [10]:
lineage_budlum = guesser_budlum.guess_lineage()

/home/niels/Documents/bread/src/bread/algo/lineage/_lineage.py:294: LineageWarning: Unable to determine parent, got exception NoCandidateParentException('No candidate parents have been found for in frame #19.'). Computation for frame #19, bud #3 skipped.
  warnings.warn(LineageWarning(f'Unable to determine parent, got exception {repr(e)}. Computation for frame #{time_id_}, bud #{bud_id} skipped.'))
/home/niels/Documents/bread/src/bread/algo/lineage/_lineage.py:294: LineageWarning: Unable to determine parent, got exception NoCandidateParentException('No candidate parents have been found for in frame #20.'). Computation for frame #20, bud #3 skipped.
  warnings.warn(LineageWarning(f'Unable to determine parent, got exception {repr(e)}. Computation for frame #{time_id_}, bud #{bud_id} skipped.'))
/home/niels/Documents/bread/src/bread/algo/lineage/_lineage.py:294: LineageWarning: Unable to determine parent, got exception NoCandidateParentException('No candidate parents have been found for i

## Evaluating accuracy of the guesses

In [11]:
lineage_truth = Lineage.from_csv('../data/colony003_lineage.csv')

In [12]:
lineage_truth

Lineage(parent_ids=array([-1, -1,  1,  2,  4,  2,  1,  3,  6,  2,  4,  5,  6,  2, 10,  9,  4,
       11,  5,  3, 12,  8,  1,  7, 14,  2,  6, 10, 15, 13,  4, 20,  3,  9,
       11, 26, 17,  5, 18,  2, 14, 19,  6, 27, 12,  3, 22, 15,  4, 31,  8,
        1, 25, 33,  5, 11, 23, 38,  6, 10,  2, 24, 16, 20,  7, 19, 21, 42,
       12,  3, 18,  9, 32, 43, 26, 13, -2, 46, 23,  4, 45,  5, 36, 28, 25,
       55, 37, 49,  6, 15, 24, 22, 77, 47, 33, 27,  8]), bud_ids=array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]), time_ids=array([  0,   0,  19,  20,  39,  40,  42,  60,  61,  62,  63,  67,  81,
        82,  84,  85,  86,  

In [13]:
accuracy(lineage_truth, lineage_expspeed, strict=False)

0.7311827956989247

In [14]:
accuracy(lineage_truth, lineage_budlum, strict=False)

0.9894736842105263

## Visualize differences

Note : some ParentID's are special codes documented in `bread.data.Lineage.SpecialParentIDs`

In [15]:
print(Lineage.SpecialParentIDs.__doc__)

Special parent IDs attributed in lineages to specify exceptions.
		
		Attributes
		----------
		PARENT_OF_ROOT : int = -1
			parent of a cell that already exists in first frame of colony
		PARENT_OF_EXTERNAL : int = -2
			parent of a cell that does not belong to the colony
		NO_GUESS : int = -3
			parent of cell for which the algorithm failed to guess
		


In [16]:
from visualize_lineages import visualize_lineages

In [17]:
visualize_lineages(lineage_truth, lineage_budlum)

ParentID (truth),ParentID (predicted),BudID,FrameID
-1,-1,1,0
-1,-1,2,0
1,1,3,19
2,2,4,20
4,4,5,39
2,2,6,40
1,1,7,42
3,3,8,60
6,6,9,61
2,2,10,62


In [18]:
visualize_lineages(lineage_truth, lineage_expspeed)

ParentID (truth),ParentID (predicted),BudID,FrameID
-1,-1,1,0
-1,-1,2,0
1,-2,3,19
2,2,4,20
4,4,5,39
2,2,6,40
1,1,7,42
3,3,8,60
6,6,9,61
2,2,10,62


## Saving predicted lineages

In [19]:
lineage_expspeed.save_csv('outputs/colony003_lineage_expspeed.csv')
lineage_budlum.save_csv('outputs/colony003_lineage_budlum.csv')